## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import optuna
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import lightgbm as lgb

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Data Load

In [4]:
# 데이터 로드
Total_train = pd.read_csv('/content/drive/MyDrive/LG_Aimers_6th/Total_train_dataset_32.csv')
Total_test = pd.read_csv('/content/drive/MyDrive/LG_Aimers_6th/Total_test_dataset_32.csv')

In [5]:
# ID 열을 제외한 특성과 타겟 변수 분리
Total_X = Total_train.drop(['임신_성공_여부', 'ID'], axis=1)
Total_y = Total_train['임신_성공_여부']

### 인코딩

In [6]:
Total_categorical_columns = [
    "시술_당시_나이",
    "배란_유도_유형",
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이"
]

In [7]:
# 모든 범주형 변수를 문자열로 변환
Total_X[Total_categorical_columns] = Total_X[Total_categorical_columns].astype(str)
Total_test[Total_categorical_columns] = Total_test[Total_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
Total_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

Total_X[Total_categorical_columns] = Total_encoder.fit_transform(Total_X[Total_categorical_columns])
Total_test[Total_categorical_columns] = Total_encoder.transform(Total_test[Total_categorical_columns])

## Modeling

In [8]:
# 데이터 분할
Total_X_train, Total_X_test, Total_y_train, Total_y_test = train_test_split(Total_X, Total_y, test_size=0.2, random_state=42)

### Total 데이터

optuna

In [9]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.5 MB/s eta 0:00:00


In [10]:
from catboost import CatBoostClassifier

In [11]:
# 목적 함수 정의
def objective(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 1000, 4000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.2),
        'depth': trial.suggest_int('depth', 4, 13),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 5),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10, log=True),
        'random_strength': trial.suggest_float('random_strength', 0, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.1, 10),
        'border_count': trial.suggest_int('border_count', 128, 300),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),

        'random_state': 42,
        'logging_level': 'Silent',
        'boosting_type': 'Plain',
        'task_type': 'GPU',  # GPU 사용 설정
    }

    model = CatBoostClassifier(**param)
    model.fit(Total_X_train, Total_y_train)

    y_pred_proba = model.predict_proba(Total_X_test)[:, 1]

    auc = roc_auc_score(Total_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-16 06:05:03,755] A new study created in memory with name: no-name-32b02d76-60fa-4820-852d-28e957a642df
[I 2025-02-16 06:05:35,661] Trial 0 finished with value: 0.735818306938326 and parameters: {'iterations': 1019, 'learning_rate': 0.090346215086567, 'depth': 7, 'min_data_in_leaf': 3, 'l2_leaf_reg': 6.894569821037426, 'random_strength': 2.8209413212701007, 'bagging_temperature': 1.9454812425174806, 'border_count': 151, 'scale_pos_weight': 5.320291000702274, 'grow_policy': 'SymmetricTree'}. Best is trial 0 with value: 0.735818306938326.
[I 2025-02-16 06:11:09,126] Trial 1 finished with value: 0.6806481926406325 and parameters: {'iterations': 2752, 'learning_rate': 0.15127512420443756, 'depth': 12, 'min_data_in_leaf': 5, 'l2_leaf_reg': 3.1102789152771044, 'random_strength': 1.6929242117637988, 'bagging_temperature': 4.06919083821419, 'border_count': 243, 'scale_pos_weight': 6.306579713816484, 'grow_policy': 'Depthwise'}. Best is trial 0 with value: 0.735818306938326.
[I 2025-0

Best trial:
  Value: 0.7397476383134949
  Params: 
    iterations: 3194
    learning_rate: 0.023354284335161152
    depth: 4
    min_data_in_leaf: 2
    l2_leaf_reg: 9.3574320146087
    random_strength: 4.381073915116191
    bagging_temperature: 1.1309218772205971
    border_count: 191
    scale_pos_weight: 1.9078201865715982
    grow_policy: SymmetricTree


.